In [21]:
import numpy as np
from scipy.stats import kurtosis, mode, skew

In [22]:
def data_to_array(archivo):
    raw_data = []
    with open(archivo) as file:
        lines = file.readlines()
        for line in lines:
            raw_data.append(np.array(line.strip().split()).astype(float))
    return raw_data

In [18]:
def caracterizacion(raw_data):
    caracteristicas = []
    for data in raw_data:
        caracteristicas.append(
            [np.mean(data),
             np.median(data),
             kurtosis(data),
             np.std(data),
             np.var(data),
             skew(data),
             mode(data)[0][0]
            ])
    data = np.array(caracteristicas)
    return data

In [28]:
raw_data = data_to_array("synthetic_control.data")
data = caracterizacion(raw_data)

print(data[111])

[  3.09182750e+01   3.08956500e+01  -1.17198040e+00   9.94367420e+00
   9.88766566e+01   1.79239057e-03   1.14926000e+01]
